In [1]:
# TEMPLATE: MySearch

from MyModules.useful_functions import *
ls_extensions = ["py", "ipynb", "md", "csv", "png"]
ms = MySearch(extensions = ls_extensions)

query = '''
select 
from 
where
'''

Execution Time: 0.01 seconds


---

In [10]:
import pandas as pd
def search(self, keywords, mode="or", file_list=None
           , case_sensitive=False):
    if not file_list:
        df = self.mfl.df_file_list
        file_path = lambda x: f"{x.dir_path}/{x.file_name}"
        df["file_path"] = df.apply(file_path, axis = 1)
        file_list = list(df["file_path"])
        
    #ifs = PowerInFileSearch(file_list, case_sensitive)
    #df_ifs = ifs.search(keywords, mode)
    #df_res = pd.merge(df_ifs, df, on="file_path")
    df_res = df
    self.mfl.show(df_res)
    
keywords = ["mdf", "mfl"]
#search(ms, keywords)
file_list = list(ms.mfl.df_file_list.file_path)
case_sensitive=False
ifs = PowerInFileSearch(file_list, case_sensitive)

In [ ]:
ifs = PowerInFileSearch(file_list=["./my_button.ipynb"])
ifs.search(keywords="test")

In [16]:
file_list=["./my_button.ipynb"]

In [17]:
file_list

['./my_button.ipynb']

### シンプルバージョン
検索文字は一つのみ。

In [39]:
import pandas as pd
import concurrent.futures

class PowerInFileSearch:

    def __init__(self, file_list):
        self.file_list = file_list

    def _search_file(self, file_path, keyword):
        hits = []
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            for line_num, line in enumerate(f, 1):
                if keyword in line:
                    hits.append((file_path, line_num, line.strip()))
        return hits

    def search(self, keyword):
        results = []
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = {executor.submit(self._search_file, file_path, keyword): file_path for file_path in self.file_list}
            for future in concurrent.futures.as_completed(futures):
                results.extend(future.result())

        df = pd.DataFrame(results, columns=["file_name", "line_number", "line_content"])
        return df



In [40]:
searcher = PowerInFileSearch(file_list)
df = searcher.search('test')
display(df.head())

,file_name,line_number,line_content
0,./my_button.ipynb,60,"""def test_func(x):\n"","
1,./my_button.ipynb,63,"""btn = MyButtonNextPrev(test_func, idx=5, min_..."
2,./my_button.ipynb,103,"""btn = MyButtons(test_func, [\""りんご\"", \""オレンジ\""..."


### バージョン2
検索文字は複数。大文字小文字の制御を追加。

In [36]:
import pandas as pd
import concurrent.futures

class PowerInFileSearch:

    def __init__(self, file_list):
        self.file_list = file_list

    def _search_file(self, file_path, keywords, flg_casesensitive):
        hits = []
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            for line_num, line in enumerate(f, 1):
                # 大文字小文字の区別が不要な場合は、行とキーワードを両方とも小文字に変換します。
                search_line = line if flg_casesensitive else line.lower()
                for keyword in keywords:
                    search_keyword = keyword if flg_casesensitive else keyword.lower()
                    if search_keyword in search_line:
                        hits.append((file_path, line_num, line.strip()))
                        break  # 同じ行に複数のキーワードがマッチしても1回のみヒットとして扱う
        return hits

    def search(self, keywords, flg_casesensitive=True):
        results = []
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = {executor.submit(self._search_file, file_path, keywords, flg_casesensitive): file_path for file_path in self.file_list}
            for future in concurrent.futures.as_completed(futures):
                results.extend(future.result())

        df = pd.DataFrame(results, columns=["file_name", "line_number", "line_content"])
        return df


### リンクを追加。

In [42]:
import pandas as pd
import concurrent.futures
import os

class PowerInFileSearch:

    def __init__(self, file_list):
        self.file_list = file_list
        self.df = None

    def _search_file(self, file_path, keywords, flg_casesensitive):
        hits = []
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            for line_num, line in enumerate(f, 1):
                search_line = line if flg_casesensitive else line.lower()
                for keyword in keywords:
                    search_keyword = keyword if flg_casesensitive else keyword.lower()
                    if search_keyword in search_line:
                        dir_path, file_name = os.path.split(file_path)
                        hits.append((file_path, dir_path, file_name, line_num, line.strip()))
                        break
        return hits

    def search(self, keywords, flg_casesensitive=True):
        results = []
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = {executor.submit(self._search_file, file_path, keywords, flg_casesensitive): file_path for file_path in self.file_list}
            for future in concurrent.futures.as_completed(futures):
                results.extend(future.result())

        self.df = pd.DataFrame(results, columns=["file_path", "dir_path", "file_name", "line_number", "line_content"])
        self.df["link"] = "<a href='" + self.df["file_path"] + "'>link</a>"

    def show(self):
        if self.df is not None:
            from IPython.display import display, HTML
            display(HTML(self.df.to_html(escape=False)))
        else:
            print("No results to display.")



In [45]:
searcher = PowerInFileSearch(file_list)
searcher.search(['test'])
display(searcher.df.head())
searcher.show()

,file_path,dir_path,file_name,line_number,line_content,link
0,./my_button.ipynb,.,my_button.ipynb,60,"""def test_func(x):\n"",",<a href='./my_button.ipynb'>link</a>
1,./my_button.ipynb,.,my_button.ipynb,63,"""btn = MyButtonNextPrev(test_func, idx=5, min_...",<a href='./my_button.ipynb'>link</a>
2,./my_button.ipynb,.,my_button.ipynb,103,"""btn = MyButtons(test_func, [\""りんご\"", \""オレンジ\""...",<a href='./my_button.ipynb'>link</a>


,file_path,dir_path,file_name,line_number,line_content,link
0,./my_button.ipynb,.,my_button.ipynb,60,"""def test_func(x):\n"",",link
1,./my_button.ipynb,.,my_button.ipynb,63,"""btn = MyButtonNextPrev(test_func, idx=5, min_idx=0, max_idx=9990)\n"",",link
2,./my_button.ipynb,.,my_button.ipynb,103,"""btn = MyButtons(test_func, [\""りんご\"", \""オレンジ\"", \""バナナ\""], flg_Vertical=False)\n"",",link


### 
前後行を確認できるように改善

In [46]:
import pandas as pd
import concurrent.futures
import os

class PowerInFileSearch:

    def __init__(self, file_list):
        self.file_list = file_list
        self.df = None

    def _search_file(self, file_path, keywords, flg_casesensitive, nrows):
        hits = []
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            lines = f.readlines()
            for line_num, line in enumerate(lines, 1):
                search_line = line if flg_casesensitive else line.lower()
                for keyword in keywords:
                    search_keyword = keyword if flg_casesensitive else keyword.lower()
                    if search_keyword in search_line:
                        dir_path, file_name = os.path.split(file_path)
                        
                        # 前後の行を含めた内容を取得
                        start_index = max(0, line_num - nrows - 1)
                        end_index = min(len(lines), line_num + nrows - 1)
                        around_lines = lines[start_index:end_index]
                        around_content = ''.join(around_lines).strip()
                        
                        hits.append((file_path, dir_path, file_name, line_num, line.strip(), around_content))
                        break
        return hits

    def search(self, keywords, flg_casesensitive=True, nrows=3):
        results = []
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = {executor.submit(self._search_file, file_path, keywords, flg_casesensitive, nrows): file_path for file_path in self.file_list}
            for future in concurrent.futures.as_completed(futures):
                results.extend(future.result())

        self.df = pd.DataFrame(results, columns=["file_path", "dir_path", "file_name", "line_number", "line_content", "around_content"])
        self.df["link"] = "<a href='" + self.df["file_path"] + "'>link</a>"

    def show(self):
        if self.df is not None:
            from IPython.display import display, HTML
            display(HTML(self.df.to_html(escape=False)))
        else:
            print("No results to display.")



In [47]:
searcher = PowerInFileSearch(file_list)
searcher.search(['test'])
display(searcher.df.head())
searcher.show()

,file_path,dir_path,file_name,line_number,line_content,around_content,link
0,./my_button.ipynb,.,my_button.ipynb,60,"""def test_func(x):\n"",","""source"": [\n ""from MyModules.useful_functi...",<a href='./my_button.ipynb'>link</a>
1,./my_button.ipynb,.,my_button.ipynb,63,"""btn = MyButtonNextPrev(test_func, idx=5, min_...","""def test_func(x):\n"",\n "" print(f\""Curr...",<a href='./my_button.ipynb'>link</a>
2,./my_button.ipynb,.,my_button.ipynb,103,"""btn = MyButtons(test_func, [\""りんご\"", \""オレンジ\""...","}\n ],\n ""source"": [\n ""btn = MyButtons...",<a href='./my_button.ipynb'>link</a>


,file_path,dir_path,file_name,line_number,line_content,around_content,link
0,./my_button.ipynb,.,my_button.ipynb,60,"""def test_func(x):\n"",","""source"": [\n ""from MyModules.useful_functions import *\n"",\n ""\n"",\n ""def test_func(x):\n"",\n "" print(f\""Current index: {x}\"")\n"",\n ""\n"",",link
1,./my_button.ipynb,.,my_button.ipynb,63,"""btn = MyButtonNextPrev(test_func, idx=5, min_idx=0, max_idx=9990)\n"",","""def test_func(x):\n"",\n "" print(f\""Current index: {x}\"")\n"",\n ""\n"",\n ""btn = MyButtonNextPrev(test_func, idx=5, min_idx=0, max_idx=9990)\n"",\n ""btn.show()\n""\n ]",link
2,./my_button.ipynb,.,my_button.ipynb,103,"""btn = MyButtons(test_func, [\""りんご\"", \""オレンジ\"", \""バナナ\""], flg_Vertical=False)\n"",","}\n ],\n ""source"": [\n ""btn = MyButtons(test_func, [\""りんご\"", \""オレンジ\"", \""バナナ\""], flg_Vertical=False)\n"",\n ""btn.show()\n""\n ]",link
